# Shonil Dabreo, s3835204, Group 62

### Loading dataset
We have already prepared the dataset in Question 1. Importing packages and displaying the dataset shape and feature values.

In [1]:
import warnings
warnings.filterwarnings("ignore")

# importing packages
import numpy as np
import pandas as pd

# Setting random seed to 999
np.random.seed(999)

# Importing THA_diamonds.csv dataset and assigning it to a variable using read_csv function
diamond = pd.read_csv('THA_diamonds.csv', decimal = '.')

# printing no of observations & features
print(diamond.shape)

# headers
diamond.columns.values

(212, 5)


array(['cut', 'color', 'depth', 'price', 'carat'], dtype=object)

In [2]:
# To display all columns in a dataframe. 
pd.set_option('display.max_columns', None) 
diamond

,cut,color,depth,price,carat
0,Good,D,63.6,low,0.44
1,Fair,F,64.2,low,0.45
2,Good,I,60.4,low,0.50
3,Good,F,56.8,low,0.45
4,Fair,F,64.3,low,0.45
...,...,...,...,...,...
207,Good,F,63.7,premium,0.96
208,Fair,D,57.5,premium,0.90
209,Fair,F,64.7,premium,0.90
210,Good,I,58.2,premium,0.93


## Discretizing the Features

We will discretize `carat` & `depth` numeric features into categorical features for successful build of model. First we will create a copy of the dataset and then using `qcut` (quantile-based discretization) for discretizing the features. 
Note that **rank** function is used to bin according to the rank and avoid including duplicate values while binning.

In [10]:
diam_copy = diamond.copy()

diam_copy['carat'] = pd.qcut(diam_copy['carat'].rank(method = 'first'), q = 3, 
                             labels = ['carat_1', 'carat_2', 'carat_3'])

diam_copy['depth'] = pd.qcut(diam_copy['depth'].rank(method = 'first'), q = 3, 
                             labels = ['depth_1', 'depth_2', 'depth_3'])

We will use `value_counts` method from **pandas** to verify we correctly implemented the binning technique.

In [11]:
diam_copy['carat'].value_counts()

carat_3    71
carat_1    71
carat_2    70
Name: carat, dtype: int64

Lets see the contents of the dataset.

In [12]:
diam_copy.head(5)

,cut,color,depth,price,carat
0,Good,D,depth_2,low,carat_1
1,Fair,F,depth_3,low,carat_1
2,Good,I,depth_1,low,carat_1
3,Good,F,depth_1,low,carat_1
4,Fair,F,depth_3,low,carat_1


We can see the category names present in the dataset after binning.

### Compute Impurity of Target Feature 

For computing impurity of `price` feature, we will use the following code from the notes. 
We will first calculate the probability distribution for each category in `price`. Then compute the impurity using **entropy** critierion.

In [19]:
probability = diam_copy['price'].value_counts(normalize = True)
price_entropy = -1 * np.sum(np.log2(probability) * probability).round(3)
price_entropy

1.716

1.716 is the entropy impurity value for the target feature `price`. 

### Determine Root Node for Desicion Tree

To determine the root node we will use the defined function from the notes. `compute_feature_information_gain` function is applied to calculate the information gain for splitting at root node.   

In [23]:
def compute_feature_information_gain(df, target, nominal_feature):
    """
    This function calculates information gain for splitting on 
    a particular descriptive feature for a given dataset
    """
    
    print('target feature:', target)
    print('nominal_feature:', nominal_feature)

    # we define two lists below:
    # entropy_list to store the entropy of each partition
    # weight_list to store the relative number of observations in each partition
    entropy_list = list()
    weight_list = list()
    
    # loop over each level of the descriptive feature
    # to partition the dataset with respect to that level and compute the entropy and the weight of the level's partition
    for level in df[nominal_feature].unique():
        df_feature_level = df[df[nominal_feature] == level]
        
        # compute impurity of target from partioned data of a level
        probability = df_feature_level['price'].value_counts(normalize = True)
        entropy_level = -1 * np.sum(np.log2(probability) * probability).round(3)
        
        entropy_list.append(round(entropy_level, 3))
        weight_level = len(df_feature_level) / len(df)
        weight_list.append(round(weight_level, 3))

    print('impurity of partitions:', entropy_list)
    print('weights of partitions:', weight_list)

    feature_remaining_impurity = np.sum(np.array(entropy_list) * np.array(weight_list))
    print('remaining impurity:', feature_remaining_impurity.round(3))
    
    information_gain = price_entropy - feature_remaining_impurity
    print('information gain:', information_gain.round(3))
    
    print('====================')

    return(information_gain)

In [24]:
for feat in diam_copy.drop(columns = 'price').columns:
    feature_info_gain = compute_feature_information_gain(diam_copy, 'price', feat)

target feature: price
nominal_feature: cut
impurity of partitions: [1.68, 1.78]
weights of partitions: [0.717, 0.283]
remaining impurity: 1.708
information gain: 0.008
target feature: price
nominal_feature: color
impurity of partitions: [1.657, 1.445, 1.833]
weights of partitions: [0.269, 0.434, 0.297]
remaining impurity: 1.617
information gain: 0.099
target feature: price
nominal_feature: depth
impurity of partitions: [1.465, 1.749, 1.74]
weights of partitions: [0.33, 0.335, 0.335]
remaining impurity: 1.652
information gain: 0.064
target feature: price
nominal_feature: carat
impurity of partitions: [-0.0, 1.406, 1.486]
weights of partitions: [0.335, 0.33, 0.335]
remaining impurity: 0.962
information gain: 0.754


It can be understood that the highest ***information gain - 0.754*** we get is for the `carat` feature which will be the split for the root node. 

**df_splits**

|split | remainder | info_gain | is_optimal|
|---|---|---|---|
|cut | 1.708 | 0.008 | False |
|color | 1.617 | 0.099 | False |
|depth | 1.652 | 0.064 | False |
|carat | 0.962 | 0.754 | True |